Q1-A

In [1]:
import numpy as np


n = int(input("Enter the dimension of the matrix: "))
# enter each row in one line
# matrix = np.array([list(map(int, input().split())) for i in range(n)])
matrix = np.array(
     [[0, 0.1, 0.5, 0.3, 0.7],
      [0.1, 0, 0.2, 0.4, 0.8],
      [0.5, 0.2, 0, 0.9, 0.6],
      [0.3, 0.4, 0.9, 0, 0.3],
      [0.7, 0.8, 0.6, 0.3, 0]]
)

convergence_criteria = 0.000001 # 10^-6


def find_eigenvectors_using_power_iteration(matrix, iterations=1000):
    x = np.array([1 for i in range(len(matrix))])   # initial guess
    for i in range(iterations):
        x_prev = x.copy()
        x = np.dot(matrix, x)
        x = x / np.linalg.norm(x)
        
        # check for convergence
        if np.abs(np.linalg.norm(x) - np.linalg.norm(x_prev)) < convergence_criteria:
            return x
    print(f"Did not converge in {iterations} iterations")
    return x


def find_top_three_nodes_for_centrality(matrix):
    # find eigenvector corresponding to largest eigenvalue
    eigenvector = find_eigenvectors_using_power_iteration(matrix)
    # find top 3 nodes
    centrality = np.abs(eigenvector)
    top_three_nodes = np.argsort(centrality)[::-1][:3]
    return top_three_nodes

print("Eigenvector corresponding to largest eigenvalue: ")
print(find_eigenvectors_using_power_iteration(matrix))
print("Top 3 nodes with the highest centrality: ")
print(find_top_three_nodes_for_centrality(matrix))

Eigenvector corresponding to largest eigenvalue: 
[0.40941021 0.38367586 0.49714097 0.44216303 0.492462  ]
Top 3 nodes with the highest centrality: 
[2 4 3]


Q1-B

In [2]:
# construct a laplacian matrix from the adjacency matrix
def construct_laplacian_matrix(matrix):
    degree_matrix = np.diag([sum(matrix[i]) for i in range(len(matrix))])
    laplacian_matrix = degree_matrix - matrix
    return laplacian_matrix

# find Fiedler vector (the second-smallest eigenvector) of the Laplacian matrix
def find_fiedler_vector(matrix):
    laplacian_matrix = construct_laplacian_matrix(matrix)
    eigenvalues, eigenvectors = np.linalg.eig(laplacian_matrix)
    fiedler_vector = eigenvectors[:, 1]
    return fiedler_vector


def find_clusters_of_friendship(matrix):
    fiedler_vector = find_fiedler_vector(matrix)
    # separate clusters based on the sign of the Fiedler vector
    cluster1 = [i for i in range(len(fiedler_vector)) if fiedler_vector[i] >= 0]
    cluster2 = [i for i in range(len(fiedler_vector)) if fiedler_vector[i] < 0]
    return cluster1, cluster2


def nullity(matrix):
    return len(matrix) - np.linalg.matrix_rank(matrix)

    
print("Laplacian matrix: ")
print(construct_laplacian_matrix(matrix))
print("Fiedler vector: ")
print(find_fiedler_vector(matrix))
print("Clusters of friendship: ")
print(find_clusters_of_friendship(matrix))

print("Nullity of the Laplacian matrix: ", nullity(construct_laplacian_matrix(matrix)))

Laplacian matrix: 
[[ 1.6 -0.1 -0.5 -0.3 -0.7]
 [-0.1  1.5 -0.2 -0.4 -0.8]
 [-0.5 -0.2  2.2 -0.9 -0.6]
 [-0.3 -0.4 -0.9  1.9 -0.3]
 [-0.7 -0.8 -0.6 -0.3  2.4]]
Fiedler vector: 
[-0.17243677 -0.31934463 -0.52996656  0.32999342  0.69175455]
Clusters of friendship: 
([3, 4], [0, 1, 2])
Nullity of the Laplacian matrix:  1


Q1-C

Q2

In [3]:
import numpy as np
import time


N = 1234
fib_matrix = np.array([[1, 1], [1, 0]])


def matrix_multiplication(matrix, n):
    # use A^2 to calculate in O(log n)
    if n == 1:
        return matrix
    else:
        half = matrix_multiplication(matrix, n//2)
        if n % 2 == 0:
            return np.dot(half, half)
        else:
            return np.dot(np.dot(half, half), matrix)

        
def matrix_multiplication_using_eigenvectors(matrix):
    # A = PDP^-1
    # A^n = PD^nP^-1
    
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    P = eigenvectors
    # D = np.diag(eigenvalues)
    P_inv = np.linalg.inv(P)
    # use matrix_multiplication to calculate D^n
    D_power_n = matrix_multiplication(np.diag(eigenvalues), N)
    result = np.dot(np.dot(P, D_power_n), P_inv)
    # if the result is in scientific notation, convert it to an integer
    result = np.array([[int(result[i][j]) for j in range(len(result[i]))] for i in range(len(result))])
    return result


print("calculating A^N using eigenvectors...")
start = time.time()
result = matrix_multiplication_using_eigenvectors(fib_matrix)
end = time.time()
print("result: ")
print(result)
print("nth fibonacci number: ", result[0][1])
print("Time taken: ", end-start, "seconds")
print()
        
print("calculating A^N using matrix multiplication in O(log n)...")
start = time.time()
result = matrix_multiplication(fib_matrix, N)
end = time.time()
print("result: ")
print(result)
print("nth fibonacci number: ", result[0][1])
print("Time taken: ", end-start, "seconds")

calculating A^N using eigenvectors...
result: 
[[562666043470786140987014004364455922830291254449518195833126492124477401145175508196761925338604020403328657564246422586816366843722311490696160201122378809106134779388840824752807502381465943413687931312079272298471628420463629239731177204418909236264697856
  347746739180371690203199316950597274580436062820877071858230443699913266674995758441600205448424256303143556282093284667455391092236800895219882852751938327474531633940082949609141922938432167022413433847350502063113724778470905985954988267613739686916259840]
 [347746739180371636859083853911763081941621737050814694700163865172647983504933264329302302907367684131685949309658958724016106344770877328109469619797017684567683215293442580847881286826315699317787146552838304231393595691378090132429003967802049357605240832
  21491930429041439743969922437502445561104086585857874681682946989729885130011725564286381734912319192872749430971881197592169100401958702836586411541551983872475472680